# 4.1.1 Running Prednet on Breakfast Clips

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,h5py,pandas,matplotlib,seaborn,jupyterlab,lab

Fri Mar 06 2020 15:57:38 

CPython 3.6.10
IPython 7.12.0

torch 1.2.0
torchvision 0.1.8
cv2 3.4.2
h5py 2.8.0
pandas 1.0.1
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.38.g6a19aca.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-76-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : 6a19aca9e16ed91aaf852f0914eb45b18ea68d92
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode
%matplotlib inline

## Set the GPU

Make sure we aren't greedy.

In [4]:
!nvidia-smi

Fri Mar  6 15:58:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:04:00.0 Off |                  N/A |
| 28%   49C    P2    75W / 250W |   9399MiB / 12196MiB |     31%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 39%   63C    P2    89W / 250W |  12037MiB / 12196MiB |      7%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


## Imports

In [6]:
from pathlib import Path

import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [11]:
# Constants to be used throughout the package
%aimport lab
import lab
%aimport lab.index
from lab.index import DIR_DATA_INT, DIR_DATA_RAW
%aimport lab.breakfast
import lab.breakfast as bk
%aimport lab.breakfast.constants
from lab.breakfast.constants import SEED
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import (DIR_BREAKFAST, 
                                 DIR_BREAKFAST_DATA, 
                                 DIR_COARSE_SEG, 
                                 DIR_FINE_SEG,
                                 DIR_BK_WEIGHTS,
                                 DIR_BK_CHECKPOINTS,
                                )
%aimport lab.breakfast.prednet
from lab.breakfast.prednet import PredNet
%aimport lab.breakfast.dataloader
from lab.breakfast.dataloader import Breakfast64DimFVDataset, BreakfastI3DFVDataset

## Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [8]:
sns.set()

## Splitting the Data

See [this](https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets/50544887#50544887) stackoverflow post.

In [29]:
ds = Breakfast64DimFVDataset()

In [74]:
np.random.seed(SEED)

ds_length = len(ds)
indices = list(range(ds_length))
n_test = 128
batch_size = 256

np.random.shuffle(indices)
train_indices, test_indices = indices[n_test:], indices[:n_test]

In [75]:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(ds, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(ds, batch_size=batch_size, sampler=test_sampler)

In [76]:
num_epochs = 5

for epoch in range(num_epochs):
    for batch_idx, (data, path) in enumerate(train_loader):
        if batch_idx is 1:
            print(Path(path[0]).stem)

24_pancake_webcam01_start_6219_seed_117
37_friedegg_webcam01_start_1041_seed_117
41_salat_webcam02_start_152_seed_117
43_pancake_cam02_start_1767_seed_117
46_tea_webcam01_start_394_seed_117


## First Pass at Training

In [77]:
%aimport lab.breakfast.prednet
from lab.breakfast.prednet import PredNet

In [78]:
num_epochs = 10
A_channels = (64, 128, 256)
R_channels = (64, 128, 256)
lr = 0.001 # if epoch < 75 else 0.0001
nt = 64 # num of time steps

In [79]:
layer_loss_weights = Variable(torch.FloatTensor([[1.], [0.], [0.]]).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())

# layer_loss_weights = Variable(torch.FloatTensor([[1.], [0.], [0.], [0.]]))
# time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
# time_loss_weights[0] = 0
# time_loss_weights = Variable(time_loss_weights)

In [80]:
model = PredNet(R_channels, A_channels, output_mode='error')
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

Using GPU.


In [81]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs //2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer

In [82]:
path_checkpoint = DIR_BK_CHECKPOINTS / 'checkpoint.tar'
path_weights = DIR_BK_WEIGHTS / 'training.pt'

In [83]:
for epoch in range(num_epochs):
    optimizer = lr_scheduler(optimizer, epoch)
    for batch_idx, (data, path) in enumerate(train_loader):
        data = Variable(data)
        errors = model(data) # batch x n_layers x nt
        loc_batch = errors.size(0)
        errors = torch.mm(errors.view(-1, nt), time_loss_weights) # batch*n_layers x 1
        errors = torch.mm(errors.view(loc_batch, -1), layer_loss_weights)
        errors = torch.mean(errors, axis=0)

        optimizer.zero_grad()
        errors.backward()
        optimizer.step()
        
        if batch_idx % 50 == 0:
            print(f'Epoch: {epoch}/{num_epochs}, step: {batch_idx}/{len(ds)//batch_size}, '
                  f'errors: {errors.data[0]}')
            torch.save({
                'epoch': epoch,
                'batch_idx' : batch_idx,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'errors': errors,
            }, str(path_checkpoint))

Epoch: 0/10, step: 0/158, errors: 1.8063452243804932
Epoch: 0/10, step: 50/158, errors: 1.7441104650497437
Epoch: 0/10, step: 100/158, errors: 1.6239502429962158
Epoch: 0/10, step: 150/158, errors: 1.5582005977630615
Epoch: 1/10, step: 0/158, errors: 1.5833419561386108
Epoch: 1/10, step: 50/158, errors: 1.514613151550293
Epoch: 1/10, step: 100/158, errors: 1.4902374744415283
Epoch: 1/10, step: 150/158, errors: 1.4473806619644165
Epoch: 2/10, step: 0/158, errors: 1.436287760734558
Epoch: 2/10, step: 50/158, errors: 1.3997976779937744
Epoch: 2/10, step: 100/158, errors: 1.3535420894622803
Epoch: 2/10, step: 150/158, errors: 1.3311622142791748
Epoch: 3/10, step: 0/158, errors: 1.3118345737457275
Epoch: 3/10, step: 50/158, errors: 1.2846627235412598
Epoch: 3/10, step: 100/158, errors: 1.2719541788101196
Epoch: 3/10, step: 150/158, errors: 1.236175298690796
Epoch: 4/10, step: 0/158, errors: 1.2455084323883057
Epoch: 4/10, step: 50/158, errors: 1.2225968837738037
Epoch: 4/10, step: 100/158, 

In [ ]:
# torch.save(model.state_dict(), str(path_weights))

## Running with I3D Data

### DataLoader et al

In [9]:
%%time
ds = BreakfastI3DFVDataset()

CPU times: user 1.61 s, sys: 8.44 s, total: 10 s
Wall time: 1min 16s


In [17]:
np.random.seed(SEED)

ds_length = len(ds)
indices = list(range(ds_length))
batch_size = 128
n_test = 128

np.random.shuffle(indices)
train_indices, test_indices = indices[n_test:], indices[:n_test]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(ds, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(ds, batch_size=batch_size, sampler=test_sampler)

In [47]:
del model, layer_loss_weights, time_loss_weights
torch.cuda.empty_cache()

### Running the Model

In [48]:
%%time

num_epochs = 100
n_layers = 4
input_size = 2048
nt = 64 # num of time steps
A_channels = tuple(input_size // (2**i) for i in range(n_layers))
R_channels = tuple(input_size // (2**i) for i in range(n_layers))
lr = 0.001 # if epoch < 75 else 0.0001

path_checkpoint = DIR_BK_CHECKPOINTS / 'i3d_checkpoint.tar'
path_weights = DIR_BK_WEIGHTS / 'i3d_training.pt'

layer_loss_weights = Variable(torch.FloatTensor([[1.]] + [[0.]]*(n_layers-1)).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())

model = PredNet(R_channels, A_channels, output_mode='error')
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()
print(model)
    
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs //2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer
    
for epoch in range(num_epochs):
    optimizer = lr_scheduler(optimizer, epoch)
    for batch_idx, (data, path) in enumerate(train_loader):
        data = Variable(data)
        errors = model(data) # batch x n_layers x nt
        loc_batch = errors.size(0)
        errors = torch.mm(errors.view(-1, nt), time_loss_weights) # batch*n_layers x 1
        errors = torch.mm(errors.view(loc_batch, -1), layer_loss_weights)
        errors = torch.mean(errors, axis=0)

        optimizer.zero_grad()
        errors.backward()
        optimizer.step()
        
#         if batch_idx % 25 == 0:
#             print(f'Epoch: {epoch}/{num_epochs}, step: {batch_idx}/{len(ds)//batch_size}, '
#                   f'train error: {errors.data[0]}')
#             torch.save({
#                 'epoch': epoch,
#                 'batch_idx' : batch_idx,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'errors': errors,
#             }, str(path_checkpoint))
            
    test_errors = []

    for data, path in test_loader:
        data = Variable(data)
        errors = model(data) # batch x n_layers x nt
        loc_batch = errors.size(0)
        errors = torch.mm(errors.view(-1, nt), time_loss_weights) # batch*n_layers x 1
        errors = torch.mm(errors.view(loc_batch, -1), layer_loss_weights)
        test_errors.append(torch.mean(errors, axis=0).item())
    
    test_error = np.mean(test_errors)
    print(f'Epoch: {epoch}/{num_epochs}, test error: {test_error}')


Using GPU.
Epoch: 0/100, test error: 1.233896255493164
Epoch: 1/100, test error: 0.8275737762451172
Epoch: 2/100, test error: 0.6741371750831604
Epoch: 3/100, test error: 0.5927122235298157
Epoch: 4/100, test error: 0.5393632650375366
Epoch: 5/100, test error: 0.5091759562492371
Epoch: 6/100, test error: 0.4874243140220642
Epoch: 7/100, test error: 0.45570212602615356
Epoch: 8/100, test error: 0.42265135049819946
Epoch: 9/100, test error: 0.4050993323326111
Epoch: 10/100, test error: 0.3980809152126312
Epoch: 11/100, test error: 0.3884073495864868
Epoch: 12/100, test error: 0.38043126463890076
Epoch: 13/100, test error: 0.37487083673477173
Epoch: 14/100, test error: 0.37232667207717896
Epoch: 15/100, test error: 0.3663807213306427
Epoch: 16/100, test error: 0.3616701364517212
Epoch: 17/100, test error: 0.3601449131965637
Epoch: 18/100, test error: 0.35761237144470215
Epoch: 19/100, test error: 0.35426682233810425
Epoch: 20/100, test error: 0.35477757453918457
Epoch: 21/100, test error:

RuntimeError: shape '[128, -1]' is invalid for input of size 88

Overfitting